In [5]:
# !pip install sentence_transformers
# !pip install konlpy

     |████████████████████████████████| 19.4 MB 8.6 MB/s eta 0:00:01
     |████████████████████████████████| 449 kB 77.4 MB/s eta 0:00:01
     |████████████████████████████████| 6.9 MB 80.1 MB/s eta 0:00:01     |█████████                       | 1.9 MB 80.1 MB/s eta 0:00:01


In [6]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [7]:
doc = """
드론 활용 범위도 점차 확대되고 있다. 최근에는 미세먼지 관리에 드론이 활용되고 있다.
서울시는 '미세먼지 계절관리제' 기간인 지난달부터 오는 3월까지 4개월간 드론에 측정장치를 달아 미세먼지 집중 관리를 실시하고 있다.
드론은 산업단지와 사업장 밀집지역을 날아다니며 미세먼지 배출 수치를 점검하고, 현장 모습을 영상으로 담는다.
영상을 통해 미세먼지 방지 시설을 제대로 가동하지 않는 업체와 무허가 시설에 대한 단속이 한층 수월해질 전망이다.
드론 활용에 가장 적극적인 소방청은 광범위하고 복합적인 재난 대응 차원에서 드론과 관련 전문인력 보강을 꾸준히 이어가고 있다.
지난해 말 기준 소방청이 보유한 드론은 총 304대, 드론 조종 자격증을 갖춘 소방대원의 경우 1,860명이다.
이 중 실기평가지도 자격증까지 갖춘 ‘드론 전문가’ 21명도 배치돼 있다.
소방청 관계자는 "소방드론은 재난현장에서 영상정보를 수집, 산악ㆍ수난 사고 시 인명수색·구조활동,
유독가스·폭발사고 시 대원안전 확보 등에 활용된다"며
"향후 화재진압, 인명구조 등에도 드론을 활용하기 위해 연구개발(R&D)을 하고 있다"고 말했다.
"""

In [8]:
okt = Okt()

tokenized_doc = okt.pos(doc)
tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])

print('품사 태깅 10개만 출력:', tokenized_doc[:10])
print('명사 추출 :', tokenized_nouns)

품사 태깅 10개만 출력: [('\n', 'Foreign'), ('드론', 'Noun'), ('활용', 'Noun'), ('범위', 'Noun'), ('도', 'Josa'), ('점차', 'Noun'), ('확대', 'Noun'), ('되고', 'Verb'), ('있다', 'Adjective'), ('.', 'Punctuation')]
명사 추출 : 드론 활용 범위 점차 확대 최근 미세먼지 관리 드론 활용 서울시 미세먼지 계절 관리제 기간 지난달 개 월간 드론 측정 장치 달 미세먼지 집중 관리 실시 드론 산업 단지 사업 밀집 지역 미세먼지 배출 수치 점검 현장 모습 영상 영상 통해 미세먼지 방지 시설 제대로 가동 업체 무허가 시설 대한 단속 한층 전망 드론 활용 가장 적극 소방청 복합 재난 대응 차원 드론 관련 전문 인력 보강 어가 지난해 말 기준 소방청 보유 드론 총 드론 조종 자격증 소방대 경우 명 이 중 실기 평가 지도 자격증 드론 전문가 명도 배치 소방청 관계자 소방 드론 재난 현장 영상 정보 수집 산악 수난 사고 시 인명 수색 구조 활동 유독가스 폭발사고 시 대원 안전 확보 등 활용 며 향후 화재 진압 인명구조 등 드론 활용 위해 연구개발 고 말


In [10]:
n_gram_range = (2, 3)

count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
candidates = count.get_feature_names_out()

print('trigram 개수 :', len(candidates))
print('trigram 다섯개만 출력 :', candidates[:5])

trigram 개수 : 222
trigram 다섯개만 출력 : ['가동 업체' '가동 업체 무허가' '가장 적극' '가장 적극 소방청' '경우 실기']


In [11]:
model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
doc_embedding = model.encode([doc])
candidate_embeddings = model.encode(candidates)

Downloading:   0%|          | 0.00/574 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/4.06k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/731 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/122 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/229 [00:00<?, ?B/s]

...

Downloading:   0%|          | 0.00/5.07M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/150 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/9.10M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/527 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [13]:
top_n = 5
distances = cosine_similarity(doc_embedding, candidate_embeddings)

keywords = [candidates[index] for index in distances.argsort()[0][-top_n :]]

print(keywords)


['드론 산업', '드론 드론 조종', '실시 드론 산업', '관리 드론 활용', '미세먼지 관리 드론']


In [16]:
# Max Sum Similiarity

def max_sum_sim(doc_embedding, candidate_embeddings, words, top_n, nr_candidates) :
    
    # 문서와 각 키워드간 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    
    # 각 키워드 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings,
                                            candidate_embeddings)
    
    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 선택
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]
    
    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf 
    candidate = None
    
    for combination in itertools.combinations(range(len(words_idx)), top_n) :
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim :
            candidate = combination
            min_sim = sim
            
            
    return [words_vals[idx] for idx in candidate]

In [17]:
# 상위 10개 키워드를 선택하고 10개 중에서 서로 가장 유사성이 낮은 5개를 선택
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)

['드론 산업 단지', '전망 드론 활용', '드론 산업', '관리 드론 활용', '미세먼지 관리 드론']

In [19]:
max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=30)

['소방 드론 재난', '자격증 드론 전문가', '월간 드론 측정', '전망 드론 활용', '미세먼지 관리 드론']

In [24]:
# Maximal Marginal Relevance

def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity) :
    
    # 문서와 각 키워드 간 유사도가 적혀진 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)
    
    # 각 키워드 간 유사도
    word_similarity = cosine_similarity(candidate_embeddings)
    
    # 문서와 가장 높은 유사도를 가진 키워드 인덱스 추출
    # 만약 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]
    
    # 가장 높은 유사도를 가진 키워드 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 .. 중략 ..]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
    
    # 최고 키워드는 이미 추출했으므로 top_n -1번만큼 아래를 반복
    # ex ) top_n = 5라면, 아래 loop는 4번 반복됨
    
    for _ in range(top_n - 1) :
        candidates_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis= 1)
        
        # MMR 계산
        mmr = (1-diversity) * candidates_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]
        
        # keywords & candidates 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)
        
    return [words[idx] for idx in keywords_idx]

In [25]:
mmr(doc_embedding, candidate_embeddings, candidates, top_n = 5, diversity = 0.2)

['미세먼지 관리 드론', '실시 드론 산업', '관리 드론 활용', '월간 드론 측정', '전망 드론 활용']

In [26]:
# diversity 값을 높게 하면 다양한 키워드 5개 생성
mmr(doc_embedding, candidate_embeddings, candidates, top_n = 5, diversity = 0.7)

['미세먼지 관리 드론', '사업 밀집', '재난 현장 영상', '산악 수난', '수치 점검']

<참고>
- https://wikidocs.net/159468

---

In [1]:
import numpy as np
import itertools

from konlpy.tag import Okt
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sentence_transformers import SentenceTransformer

In [8]:
# 문서 keyword 추출 함수화

# Max Sum Similiarity
#max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n=5, nr_candidates=10)
# Maximal Marginal Relevance

def mmr(doc_embedding, candidate_embeddings, words, top_n, diversity) :
    
    # 문서와 각 키워드 간 유사도가 적혀진 리스트
    word_doc_similarity = cosine_similarity(candidate_embeddings, doc_embedding)
    
    # 각 키워드 간 유사도
    word_similarity = cosine_similarity(candidate_embeddings)
    
    # 문서와 가장 높은 유사도를 가진 키워드 인덱스 추출
    # 만약 2번 문서가 가장 유사도가 높았다면
    # keywords_idx = [2]
    keywords_idx = [np.argmax(word_doc_similarity)]
    
    # 가장 높은 유사도를 가진 키워드 인덱스를 제외한 문서의 인덱스들
    # 만약, 2번 문서가 가장 유사도가 높았다면
    # ==> candidates_idx = [0, 1, 3, 4, 5, 6, 7, 8, 9, 10 .. 중략 ..]
    candidates_idx = [i for i in range(len(words)) if i != keywords_idx[0]]
    
    # 최고 키워드는 이미 추출했으므로 top_n -1번만큼 아래를 반복
    # ex ) top_n = 5라면, 아래 loop는 4번 반복됨
    
    for _ in range(top_n - 1) :
        candidates_similarities = word_doc_similarity[candidates_idx, :]
        target_similarities = np.max(word_similarity[candidates_idx][:, keywords_idx], axis= 1)
        
        # MMR 계산
        mmr = (1-diversity) * candidates_similarities - diversity * target_similarities.reshape(-1, 1)
        mmr_idx = candidates_idx[np.argmax(mmr)]
        
        # keywords & candidates 업데이트
        keywords_idx.append(mmr_idx)
        candidates_idx.remove(mmr_idx)
        
    return [words[idx] for idx in keywords_idx]
# mmr(doc_embedding, candidate_embeddings, candidates, top_n = 5, diversity = 0.7)

def max_sum_sim(doc_embedding, candidate_embeddings, candidates, top_n, nr_candidates) :
    
    # 문서와 각 키워드간 유사도
    distances = cosine_similarity(doc_embedding, candidate_embeddings)
    
    # 각 키워드 간의 유사도
    distances_candidates = cosine_similarity(candidate_embeddings,
                                            candidate_embeddings)
    
    # 코사인 유사도에 기반하여 키워드들 중 상위 top_n개의 단어를 선택
    words_idx = list(distances.argsort()[0][-nr_candidates:])
    words_vals = [candidates[index] for index in words_idx]
    distances_candidates = distances_candidates[np.ix_(words_idx, words_idx)]
    
    # 각 키워드들 중에서 가장 덜 유사한 키워드들간의 조합을 계산
    min_sim = np.inf 
    candidate = None
    
    for combination in itertools.combinations(range(len(words_idx)), top_n) :
        sim = sum([distances_candidates[i][j] for i in combination for j in combination if i != j])
        if sim < min_sim :
            candidate = combination
            min_sim = sim
            
            
    return [words_vals[idx] for idx in candidate]

def get_embeddings(doc) :
    
    okt = Okt()
    # word tokenizing
    tokenized_doc = okt.pos(doc)
    tokenized_nouns = ' '.join([word[0] for word in tokenized_doc if word[1] == 'Noun'])
    
    n_gram_range = (2, 3) # ngram 범위 설정
    count = CountVectorizer(ngram_range=n_gram_range).fit([tokenized_nouns])
    candidates = count.get_feature_names_out()
    
    # model
    model = SentenceTransformer('sentence-transformers/xlm-r-100langs-bert-base-nli-stsb-mean-tokens')
    doc_embedding = model.encode([doc])
    candidate_embeddings = model.encode(candidates)
    
    return doc_embedding, candidate_embeddings, candidates

def get_doc_keyword(doc,mode,top_n, nr_candidates=15, diversity=0.7) :
    doc_embedding, candidate_embeddings, candidates = get_embeddings(doc)
    if mode == 'max_sum_sim' :
        return max_sum_sim(doc_embedding, candidate_embeddings, candidates,top_n=top_n, nr_candidates=nr_candidates)
    else :
        return mmr(doc_embedding, candidate_embeddings, candidates, top_n = top_n, diversity=diversity)
    
# 뉴스 기사 가져와서 적용해보기 
article = '''정부는 오미크론 변이 유행이 정점을 향해 가고 있다면서, 이번 유행의 정점이 코로나19에 대응하는 마지막 큰 위기가 될 수 있다고 16일 전망했다.
이런 가운데 정부는 사회적 거리두기 조정을 위해 이날부터 본격적으로 일상회복지원위원회 의견 수렴에 나선다. 정부는 오는 21일부터 적용할 새로운 거리두기 조치를 18일 발표할 예정이다.
손영래 보건복지부 중앙사고수습본부 사회전략반장은 16일 중앙재난안전대책본부 브리핑에서 "정점이 예측대로 형성되면서 의료체계를 준비된 범위에서 대응할 수 있다면, 이번 위기가 코로나19 전반 대응 과정에서 가장 마지막의 큰 위기가 될 것으로 본다"고 말했다.
이날 0시 기준 신규확진자는 40만741명으로 처음으로 40만명을 넘어섰다. 위중증 환자도 1천244명으로 최다치를 기록했다. 전날 하루 사망자는 164명이다.
손 반장은 "오미크론 유행은 이제 정점을 향해 확진자 발생이 최대치로 증가하고 있는 상황"이라며 "금주 또는 늦어도 다음 주 정도가 이번 유행의 정점"이라고 설명했다.
정부는 복수의 연구기관 분석을 종합해 유행 정점이 16∼22일 형성되고, 정점에서 신규 확진자는 일평균 31만6천∼37만2천명 나올 수 있다고 전망했다.
하루 확진자 수가 40만명을 넘었지만, 최근 1주일 평균으로는 34만5천242명이다. 손 반장은 유행 정점에서 신규확진자가 최대 37만2천명 발생한다는 예측도 '일평균'임을 유의해달라고 당부했다.'''    

print('MAX_SUM_SIM Result:', get_doc_keyword(article,mode='max_sum_sim',top_n= 7))
print('MMR Result:',get_doc_keyword(article,mode='mmr',top_n= 7))

MAX_SUM_SIM Result: ['가장 마지막 위기', '체계 준비 범위', '진자 수가 최근', '전반 대응 과정', '준비 범위 대응', '마지막 위기 전망', '수가 최근 주일']
MMR Result: ['대응 마지막 위기', '정부 복수 연구기관', '신규 진자', '사회 전략', '수렴 정부', '예측 형성 의료', '위원회 의견 수렴']
